In [13]:
import librosa
import os
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
#importing all functions in the preprocess.py file 
#from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical


In [14]:
DATA_PATH = "C:/Users/apt/Desktop/Data/TF/"

In [15]:
#Input: Folder Path
# Output: Tuple (Label, Indices of the labels, one-hot encoded labels)
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)

In [16]:
def prepare_dataset(path=DATA_PATH):
    labels, _, _ = get_labels(path)
    data = {}
    for label in labels:
        data[label] = {}
        data[label]['path'] = [path  + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]

        vectors = []

        for wavfile in data[label]['path']:
            wave, sr = librosa.load(wavfile, mono=True, sr=None)
            # Downsampling
            wave = wave[::3]
            mfcc = librosa.feature.mfcc(wave, sr=16000)
            vectors.append(mfcc)

        data[label]['mfcc'] = vectors

    return data

In [17]:
def load_dataset(path=DATA_PATH):
    data = prepare_dataset(path)

    dataset = []

    for key in data:
        for mfcc in data[key]['mfcc']:
            dataset.append((key, mfcc))

    return dataset[:100]

In [18]:
# Handy function to convert wav2mfcc
def wav2mfcc(file_path, max_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

In [19]:
def save_data_to_array(path=DATA_PATH, max_len=11):
    labels, _, _ = get_labels(path)

    for label in labels:
        # Init mfcc vectors
        mfcc_vectors = []

        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
            mfcc = wav2mfcc(wavfile, max_len=max_len)
            mfcc_vectors.append(mfcc)
        np.save(label + '.npy', mfcc_vectors)

In [20]:
def get_train_test(split_ratio=0.6, random_state=42):
    # Get available labels
    labels, indices, _ = get_labels(DATA_PATH)

    # Getting first arrays
    X = np.load(labels[0] + '.npy')
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load(label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)

In [24]:
# Second dimension of the feature is dim2
feature_dim_2 = 11
# Save data to array file first
save_data_to_array(max_len=feature_dim_2)
print("The data is saved in NPY")
# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 10
batch_size = 100
verbose = 1
num_classes = 3

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model






Saving vectors of label - 'happy': 100%|███████████████████████████████████████████| 1742/1742 [00:25<00:00, 68.48it/s]


The data is saved in NPY


In [25]:
# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][np.argmax(model.predict(sample_reshaped))]

In [26]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Train on 3112 samples, validate on 2076 samples
Epoch 1/10
3112/3112 [==============================] - ETA: 30s - loss: 2.9198 - acc: 0.38 - ETA: 18s - loss: 3.4224 - acc: 0.37 - ETA: 13s - loss: 3.4628 - acc: 0.34 - ETA: 11s - loss: 3.5701 - acc: 0.33 - ETA: 9s - loss: 3.2585 - acc: 0.3360 - ETA: 8s - loss: 2.9646 - acc: 0.341 - ETA: 7s - loss: 2.7293 - acc: 0.351 - ETA: 7s - loss: 2.5420 - acc: 0.356 - ETA: 6s - loss: 2.4015 - acc: 0.360 - ETA: 5s - loss: 2.2734 - acc: 0.369 - ETA: 5s - loss: 2.1743 - acc: 0.372 - ETA: 5s - loss: 2.0801 - acc: 0.374 - ETA: 4s - loss: 2.0054 - acc: 0.376 - ETA: 4s - loss: 1.9345 - acc: 0.388 - ETA: 4s - loss: 1.8758 - acc: 0.393 - ETA: 3s - loss: 1.8330 - acc: 0.394 - ETA: 3s - loss: 1.7937 - acc: 0.395 - ETA: 3s - loss: 1.7474 - acc: 0.403 - ETA: 3s - loss: 1.7055 - acc: 0.412 - ETA: 2s - loss: 1.6731 - acc: 0.417 - ETA: 2s - loss: 1.6420 - acc: 0.418 - ETA: 2s - loss: 1.6128 - acc: 0.423 - ETA: 1s - loss: 1.5870 - acc: 0.426 - ETA: 1s - loss: 1.559

3112/3112 [==============================] - ETA: 5s - loss: 0.4024 - acc: 0.850 - ETA: 5s - loss: 0.3763 - acc: 0.860 - ETA: 5s - loss: 0.3458 - acc: 0.866 - ETA: 5s - loss: 0.3268 - acc: 0.882 - ETA: 5s - loss: 0.3061 - acc: 0.892 - ETA: 5s - loss: 0.2847 - acc: 0.898 - ETA: 4s - loss: 0.2663 - acc: 0.902 - ETA: 4s - loss: 0.2543 - acc: 0.907 - ETA: 4s - loss: 0.2518 - acc: 0.907 - ETA: 4s - loss: 0.2635 - acc: 0.905 - ETA: 4s - loss: 0.2604 - acc: 0.906 - ETA: 3s - loss: 0.2560 - acc: 0.909 - ETA: 3s - loss: 0.2588 - acc: 0.910 - ETA: 3s - loss: 0.2527 - acc: 0.912 - ETA: 3s - loss: 0.2544 - acc: 0.911 - ETA: 3s - loss: 0.2489 - acc: 0.914 - ETA: 2s - loss: 0.2518 - acc: 0.912 - ETA: 2s - loss: 0.2527 - acc: 0.912 - ETA: 2s - loss: 0.2496 - acc: 0.912 - ETA: 2s - loss: 0.2471 - acc: 0.913 - ETA: 2s - loss: 0.2468 - acc: 0.913 - ETA: 1s - loss: 0.2525 - acc: 0.911 - ETA: 1s - loss: 0.2590 - acc: 0.909 - ETA: 1s - loss: 0.2608 - acc: 0.907 - ETA: 1s - loss: 0.2580 - acc: 0.908 - ETA: 

In [27]:
print(predict('C:/Users/apt/Desktop/Data/TF/cat/0e5193e6_nohash_0.wav', model=model))

cat
